In [2]:
import pandas as pd
import ast
import numpy as np

In [195]:
path = 'historical.csv'
df = pd.read_csv(path, index_col=0)[['premise', 'hypothesis', 'prompt', 'rationale']]

In [196]:
df['hypothesis'] = df['hypothesis'].apply(ast.literal_eval)

In [197]:
df.rationale = df.rationale.astype(str)
df['split'] = df.rationale.apply(lambda x: x.lower().split('most likely answer'))
df.split = df.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [198]:
df.head(1)

,premise,hypothesis,prompt,rationale,split
0,The morbidly obese man with unkempt facial hai...,"[french fries, enjoy, nausea, satisfaction, gas]",Questions: The morbidly obese man with unkempt...,1. The most likely answer is **(D) satisfactio...,is **(d) satisfaction**. the question is base...


In [199]:
# get correct index
df['correct_index'] = [{} for _ in range(len(df))]
for index, row in df.iterrows():
    for choice in row['hypothesis']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice.lower())

In [200]:
df['LLM_answer'] = df.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [201]:
df[df.LLM_answer.isna()]

,premise,hypothesis,prompt,rationale,split,correct_index,LLM_answer


In [202]:
# read paper's answer
paper = pd.read_csv('paper.csv', index_col=0)

In [203]:
paper['premise'] = paper.input.str.split('\n').apply(lambda x: x[0])
paper.set_index('premise', inplace=True)
paper.index = paper.index.str.strip()

In [204]:
df.set_index('premise', inplace=True)

In [205]:
paper['my_label'] = df['LLM_answer']

In [206]:
sum(paper.label == paper.my_label) / len(paper)

0.8074119700236115

In [207]:
df.to_csv('historical - full.csv')

In [107]:
TYPE = 'historical'
temp = pd.read_csv(f'{TYPE} - full.csv')

import re
temp[~temp['rationale'].apply(lambda x: bool(re.match(r'\d+\.', x.strip())))]

,premise,hypothesis,prompt,rationale,split,correct_index,LLM_answer
166,The child was taken against its will and what ...,"['for or', 'away from', 'being for', 'because'...",Questions: The child was taken against its wil...,"\nThe most likely answer to the question ""The ...",6th century and spread to europe and asia thro...,{'because': 1134},because
2164,"People often have to take medicine, they tend ...","['kitchen', ""doctor's office"", 'hospital', 'ba...","Questions: People often have to take medicine,...","\nThe most likely answer to the question ""Peop...",to the question 'people keep their teddy bear...,"{'kitchen': 266, 'bathroom': 281}",kitchen
2729,How is a soldier likely to travel?,"['carriage', 'armored car', 'trenches', 'tent'...","Questions: How is a soldier likely to travel?,...","The most likely answer to the question ""How is...",to the question 'how does one decide if insti...,{'tent': 976},tent
4368,Where would you put your change if you plan to...,"['backseat of car', 'purse', 'pocket', 'jar', ...",Questions: Where would you put your change if ...,"\nThe most likely answer to the question ""Wher...",is **(a) stores along beach**. stores along b...,{'pocket': 524},pocket
4453,Dan found a bunch of ball bearings behind a wa...,"['high quality motor', 'skatepark', 'machines'...",Questions: Dan found a bunch of ball bearings ...,"\nThe most likely answer to the question ""Dan ...",to the question 'contemplating a concept can ...,{'can': 42},can
6166,where do you typically buy apples?,"['farmers market', 'table', 'grocery store', '...","Questions: where do you typically buy apples?,...","The most likely answer to the question ""Where ...",to the question 'where do you buy the freshes...,"{'farmers market': 66, 'grocery store': 488, '...",farmers market
7332,Where is the last place food is before being e...,"['kitchen', 'plate', 'dinning', 'stomach', 'pa...",Questions: Where is the last place food is bef...,"\nThe most likely answer to the question ""Wher...",is **(d) body**. the flu is a contagious resp...,{'stomach': 608},stomach
7841,"The Mormon owned steak house was always busy, ...","['utah', 'nebraska', 'the sun', 'new york', 's...",NaN,"\nThe most likely answer to the question ""The ...",is **(e) new mexico**. the canadian was trave...,{'utah': 198},utah
8423,The peaceful nation was scared of their neighb...,"['violent', 'warring', 'eferendum because thei...",NaN,"\nThe most likely answer to the question ""The ...","7 french presidential election, some voters ch...",{'violent': 1152},belligerent
8868,What high rise is often made of windows?,"['warehouse', ""friend's house"", 'building', 's...",NaN,"The most likely answer to the question ""What h...",is **(d) match**. this is a term that refers ...,{'wall': 338},building
